In [2]:
import pandas as pd
import numpy as np
import gender_guesser.detector as gender
data = pd.read_csv("Data/Salaries.csv")


/opt/anaconda3/envs/myenv/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3166: DtypeWarning: Columns (3,4,5,6,12) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
# dropping columns with only useless or NaN values and dropping rows with names that were not provided
data= data.drop(['Notes', 'Agency', 'Status', 'Id', 'Benefits', 'Year'], axis=1)
data = data[data['EmployeeName'] != 'Not provided']

#converting column BasePay to type float and removing BasePay that is negative
data.dropna(subset = ['BasePay'], inplace = True)
data["BasePay"] = data.BasePay.astype(float)
data = data[data['BasePay'] >= 0]

In [4]:
# checking if there are any missing values
for col in data.columns:
    pct_missing = np.mean(data[col].isnull())
    print('{} - {}%'.format(col, round(pct_missing*100)))

EmployeeName - 0%
JobTitle - 0%
BasePay - 0%
OvertimePay - 0%
OtherPay - 0%
TotalPay - 0%
TotalPayBenefits - 0%


In [5]:
# splitting EmployeeName into first and last name, keeping first names as Name
first_name = data.EmployeeName.str.split(expand=True)
first_name = first_name.drop([1,2,3,4,5], axis=1)
data['Name'] = first_name
data = data.drop(['EmployeeName'], axis=1)

# moving column 'Name' to front of dataframe
data = data[['Name'] + [ col for col in data.columns if col != 'Name' ]]


In [6]:
# guessing gender based on name
d = gender.Detector(case_sensitive=False)
gender_list = []
def guess_gender(name):
    gender_list.append(d.get_gender(name))

name_list = data['Name'].to_list()
for i in name_list:
    guess_gender(i)

# creating new column in dataframe for 'Gender'
data['Gender'] = gender_list


In [7]:
# dropping ambiguous genders to ensure accuracy
data = data[data['Gender'] != 'unknown']
data = data[data['Gender'] != 'mostly_female']
data = data[data['Gender'] != 'mostly_male']
data = data[data['Gender'] != 'andy']


In [8]:
# making sure there are only strictly female and strictly male names
data["Gender"].value_counts()

male      69552
female    50236
Name: Gender, dtype: int64

In [9]:
# dropping job titles with less than 50 occurances
counts = data['JobTitle'].value_counts()

data = data[~data['JobTitle'].isin(counts[counts < 50].index)]

data['JobTitle'].value_counts()

Transit Operator            5327
Special Nurse               3305
Registered Nurse            2763
Police Officer 3            2198
Firefighter                 2103
                            ... 
HEALTH WORKER I               50
TESTING TECHNICIAN            50
Social Work Supervisor        50
Utility Analyst               50
IS Prg Analyst-Principal      50
Name: JobTitle, Length: 415, dtype: int64

In [10]:
# find average total pay based on gender
female_salary = data[data['Gender'] =='female']
male_salary = data[data['Gender'] == 'male' ]

print('the average total pay for women is: $', female_salary['TotalPay'].mean())

print('the average total pay for men is: $',male_salary['TotalPay'].mean())



the average total pay for women is: $ 65671.19191140805
the average total pay for men is: $ 83809.87672083464


In [11]:
# creating separate dfs according to gender, grouped by Job Title
# new dfs only include BasePay and Total Pay
f_job_salaries = female_salary.groupby(['JobTitle']).mean()
f_job_salaries = f_job_salaries.drop(['TotalPayBenefits'], axis = 1)



m_job_salaries = male_salary.groupby(['JobTitle']).mean()
m_job_salaries = m_job_salaries.drop(['TotalPayBenefits'], axis = 1)

m_job_salaries.head()
f_job_salaries.head()

,BasePay,TotalPay
JobTitle,,
ACCOUNT CLERK,42556.832391,43441.577391
ADMINISTRATIVE ANALYST,64312.000513,64543.497949
AIRPORT POLICE SERVICES AIDE,49780.150345,57998.895000
ASR Senior Office Specialist,60577.726512,62563.650930
ASSISTANT ENGINEER,80079.541111,80689.304815


In [12]:

salary_by_gender = pd.merge(f_job_salaries, m_job_salaries, on='JobTitle')
salary_by_gender.columns = ['FemaleBasePay', 'FemaleTotalPay', 'MaleBasePay', 'MaleTotalPay']
salary_by_gender.head()

,FemaleBasePay,FemaleTotalPay,MaleBasePay,MaleTotalPay
JobTitle,,,,
ACCOUNT CLERK,42556.832391,43441.577391,46213.987333,46650.336667
ADMINISTRATIVE ANALYST,64312.000513,64543.497949,64920.955294,65811.110000
AIRPORT POLICE SERVICES AIDE,49780.150345,57998.895000,52180.554167,61626.364688
ASR Senior Office Specialist,60577.726512,62563.650930,61041.301111,62685.674444
ASSISTANT ENGINEER,80079.541111,80689.304815,83160.205227,84712.798864


In [13]:
# merging the two dfs together to compare pay differenes between gender for the same job
salary_by_gender['BasePayProp']= (salary_by_gender['FemaleBasePay'] *100 )/ salary_by_gender['MaleBasePay']
salary_by_gender['TotalPayProp'] = (salary_by_gender['FemaleTotalPay'] *100 )/ salary_by_gender['MaleTotalPay']
salary_by_gender.head()



,FemaleBasePay,FemaleTotalPay,MaleBasePay,MaleTotalPay,BasePayProp,TotalPayProp
JobTitle,,,,,,
ACCOUNT CLERK,42556.832391,43441.577391,46213.987333,46650.336667,92.086476,93.121680
ADMINISTRATIVE ANALYST,64312.000513,64543.497949,64920.955294,65811.110000,99.062006,98.073863
AIRPORT POLICE SERVICES AIDE,49780.150345,57998.895000,52180.554167,61626.364688,95.399812,94.113770
ASR Senior Office Specialist,60577.726512,62563.650930,61041.301111,62685.674444,99.240556,99.805341
ASSISTANT ENGINEER,80079.541111,80689.304815,83160.205227,84712.798864,96.295507,95.250430


In [31]:
#change gender to binary variables
# from sklearn.preprocessing import LabelEncoder
# labelencoder = LabelEncoder()

# data['Gender'] = labelencoder.fit_transform(data['Gender'])


<class 'pandas.core.frame.DataFrame'>
Int64Index: 99472 entries, 22 to 148653
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Name              99472 non-null  object 
 1   JobTitle          99472 non-null  object 
 2   BasePay           99472 non-null  float64
 3   OvertimePay       99472 non-null  object 
 4   OtherPay          99472 non-null  object 
 5   TotalPay          99472 non-null  float64
 6   TotalPayBenefits  99472 non-null  float64
 7   Gender            99472 non-null  float64
dtypes: float64(4), object(4)
memory usage: 6.8+ MB


In [34]:
# salary_by_gender.to_csv('salary_by_gender.csv')
# female_salary.to_csv('female_salary.csv')
# male_salary.to_csv('male_salary.csv')
data.to_csv('full_table.csv')